##### Author: Sailesh Chauhan
##### Date  : 01/08/2021
##### Title : Twitter live tweet streaming to Azure Event Hub and sentiment analysis using TextBlob library

##### Importing required Libraries

In [0]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import nltk
from textblob import TextBlob, Word, Blobber
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data] /root/nltk_data...
[nltk_data] Package averaged_perceptron_tagger is already up-to-
[nltk_data] date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data] Package brown is already up-to-date!
Out[13]: True

##### Method for analyzing text received in twitter stream.

In [0]:
def getblobsentiment(text): 
  blob = TextBlob(text)
  i=0.00
  for sentence in blob.sentences:
      i=i+sentence.sentiment.polarity

  if i >0: 
    return 1
  elif i<0:
    return -1
  else:
    return 0

  
async def run(text):
  producer = EventHubProducerClient.from_connection_string(conn_str="Endpoint=sb://wstwitteranalytics.servicebus.windows.net/;SharedAccessKeyName=sapehtwitter;SharedAccessKey=pGu2nogyQwnSBP82B2GUxlpSWFWLBtpmNkY7IT4Y4RE=;EntityPath=ehtwitteranalytics", eventhub_name="ehtwitteranalytics")
  async with producer:
    event_data_batch = await producer.create_batch()
    event_data_batch.add(EventData(text))
    await producer.send_batch(event_data_batch)  


##### Running other notebook required for getting secretKeys variable

In [0]:
%run ./environment

##### Class listener inhereting from StreamListener class. It has three methods on_data,on_error.

In [0]:
class listener(StreamListener):
  def on_data(self, data):
    loop = asyncio.get_event_loop()
    t=json.loads(data)
            
    d1 = {"text":t["text"] ,"sentiment": getblobsentiment(t["text"]) ,"created_at" :t["created_at"]}
    print (type(data))     
    loop.run_until_complete(run(json.dumps(d1)))
    print(d1)
    
    return(True)
   
  def on_error(self, status):
    print(status)
    
consumer_key=secretKeys['consumer_key']
consumer_secret=secretKeys['consumer_secret']
acess_token=secretKeys['acess_token']
acess_token_secret=secretKeys['acess_token_secret']

auth = OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(acess_token, acess_token_secret)

twitterStream = Stream(auth, listener()) 
twitterStream.filter(track=["vaccine"])

<class 'str'>
{'text': "RT @KevinKileyCA: Every step of the way, California has imposed maximum government control: lockdowns, schools, masks, the vaccine. Where's…", 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': 'RT @SutonaShari: Why do y’all want the vaccine not to work so damn bad???? That type of energy is evil.', 'sentiment': -1, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': 'RT @JohnGoliath82: Jirre, what 5G vaccine did that bra from China take...', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': 'RT @forwardarc: Florida is the new national epicenter for the virus, accounting for a fifth of all new cases in the U.S. \nMeanwhile, Trump…', 'sentiment': 1, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': 'RT @SteveSchmidtSES: 👇. The people responsible for stoking anxiety, misinformation,and confusion to the American people about a vaccine tha…', 'sentiment': 1, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': 'RT @Khunying_Joei: ก็โชว์ vaccine passport มาสิคะ ทุกอย่างจะได้จบ', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': 'RT @13abble: แนะนำให้แสดง Passport vaccine นะคะ โชว์กางออกมาประจักษ์เลยค่ะ และเรียกร้องไปถึงภาครัฐค่ะ เพราะไม่ว่าจะคุณ หรือจะใครมีสิทธิ์และ…', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': 'RT @disclosetv: NOW - Hundreds of thousands are taking to the streets in Paris, Nice, Montpellier, Nantes, Strasbourg, Reims, Toulouse, Mar…', 'sentiment': 1, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': '@george_h__ @brmdlucas @PoliticsForAlI The vaccine hasn’t even been around for a year. Who knows what the long term side effects are', 'sentiment': -1, 'created_at': 'Sun Aug 01 10:43:34 +0000 2021'}
<class 'str'>
{'text': 'RT @Marisaann97: Y’all do Coke every weekend but scared of the vaccine', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': 'RT @hotmailneverdie: แปะ vaccine passport ก็จบเลยค่ะ', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': 'RT @teapatisserie: คุณชัญญ่าโชว์ vaccine certificate เลยค่ะ เอาให้ชาวเน็ตหน้าหงายไปเลยค่ะว่าไม่ได้ฉีดในไทยสักหน่อย บินไปฉีดต่างประเทศจริงๆ…', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': '@orwellhamster @ClarkeMicah Nah, like I said, a steady voice. Not panicking about the virus or the vaccine', 'sentiment': 1, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': 'RT @itslfi: บ้านหรืออเมกาคะ? Igไม่มีอะไรเกี่ยวกับอดมริกาเลย จริงๆง่ายมากแค่โชว์หลักฐานเดินทาง ใบฉีด vaccine visa ต่างๆ โคตรง่าย แต่ทำไมไม่…', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': 'RT @AvrahamCooperMD: Lung Lung + Lung +\n vaccine COVID \n ⬇️…', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': 'RT @Shepherd10000: Viva La Resistance https://t.co/cxXgRU4vo0', 'sentiment': 0, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': 'RT @disclosetv: NOW - Hundreds of thousands are taking to the streets in Paris, Nice, Montpellier, Nantes, Strasbourg, Reims, Toulouse, Mar…', 'sentiment': 1, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': "“SAGE warns doomsday scenario is 'realistic possibility' and UK's vaccine roll-out may even speed up mutant strain'… https://t.co/iGwju1YM3B", 'sentiment': 1, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': 'RT @TheMarieOakes: Massive protest today in Nice, France against the Macron government domestic vaccine pass.\n https://t.co/EcdT8STe6h', 'sentiment': 1, 'created_at': 'Sun Aug 01 10:43:35 +0000 2021'}
<class 'str'>
{'text': '#Congress on Sunday questioned the #Shortage of #Vaccines in the country amid claims by the government that 47 cror… https://t.c